In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, roc_curve, recall_score, auc, roc_auc_score

!ls ../results

1.jpg
berk.csv
berk-test.csv
GoogleLenetv2.png
GoogLeNet_binary61_feature_imagenet_all.npy
GoogLeNet_binary61_feature_imagenet_test7600.npy
googlenet_weights.npy
logs
metrics
models
myRAP_labels_pd.csv
OEDCWPAL_PETA_features_all.pickle
PA-100K_labels_pd.csv
partGoogleLenet.png
PETA_coarse_to_fine_labels_pd.csv
PETA.csv
PETA_labels_pd.csv
PETA_lowerBody_labels_pd.csv
PETA_ratio_positive_samples_for_attributes.json
predictions
RAP_labels_pd.csv
RAP_partion.npy
RAP_partion.txt
README.md
relation_array.npy
samples
state_transition_matrix.npy
test_predictions.csv
title.csv


# Preparing attributes_list & labels

In [3]:
import scipy.io as sio

data = sio.loadmat('/home/anhaoran/data/pedestrian_attributes_RAP/RAP_annotation/RAP_annotation.mat')['RAP_annotation']
attributes_list = []
for i in range(data['attribute_eng'][0][0].shape[0]):
    attributes_list.append(data['attribute_eng'][0][0][i][0][0])
print(attributes_list)
attributes_list_chinese = []
for i in range(data['attribute_chinese'][0][0].shape[0]):
    attributes_list_chinese.append(data['attribute_chinese'][0][0][i][0][0])
print(attributes_list_chinese)

['Female', 'AgeLess16', 'Age17-30', 'Age31-45', 'BodyFat', 'BodyNormal', 'BodyThin', 'Customer', 'Clerk', 'hs-BaldHead', 'hs-LongHair', 'hs-BlackHair', 'hs-Hat', 'hs-Glasses', 'hs-Muffler', 'ub-Shirt', 'ub-Sweater', 'ub-Vest', 'ub-TShirt', 'ub-Cotton', 'ub-Jacket', 'ub-SuitUp', 'ub-Tight', 'ub-ShortSleeve', 'lb-LongTrousers', 'lb-Skirt', 'lb-ShortSkirt', 'lb-Dress', 'lb-Jeans', 'lb-TightTrousers', 'shoes-Leather', 'shoes-Sport', 'shoes-Boots', 'shoes-Cloth', 'shoes-Casual', 'attach-Backpack', 'attach-SingleShoulderBag', 'attach-HandBag', 'attach-Box', 'attach-PlasticBag', 'attach-PaperBag', 'attach-HandTrunk', 'attach-Other', 'action-Calling', 'action-Talking', 'action-Gathering', 'action-Holding', 'action-Pusing', 'action-Pulling', 'action-CarrybyArm', 'action-CarrybyHand', 'faceFront', 'faceBack', 'faceLeft', 'faceRight', 'occlusionLeft', 'occlusionRight', 'occlusionUp', 'occlusionDown', 'occlusion-Environment', 'occlusion-Attachment', 'occlusion-Person', 'occlusion-Other', 'up-Black

In [4]:
low_level = [11]
mid_level = [9,10,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42]
high_level = [0,1,2,3,4,5,6,7,8,43,44,45,46,47,48,49,50]
attributes_list_hiar = list(np.array(attributes_list)[list(np.hstack((low_level, mid_level, high_level)))])
print("The total attributes labeled: ", len(attributes_list_hiar))
print(attributes_list_hiar)

The total attributes labeled:  51
['hs-BlackHair', 'hs-BaldHead', 'hs-LongHair', 'hs-Hat', 'hs-Glasses', 'hs-Muffler', 'ub-Shirt', 'ub-Sweater', 'ub-Vest', 'ub-TShirt', 'ub-Cotton', 'ub-Jacket', 'ub-SuitUp', 'ub-Tight', 'ub-ShortSleeve', 'lb-LongTrousers', 'lb-Skirt', 'lb-ShortSkirt', 'lb-Dress', 'lb-Jeans', 'lb-TightTrousers', 'shoes-Leather', 'shoes-Sport', 'shoes-Boots', 'shoes-Cloth', 'shoes-Casual', 'attach-Backpack', 'attach-SingleShoulderBag', 'attach-HandBag', 'attach-Box', 'attach-PlasticBag', 'attach-PaperBag', 'attach-HandTrunk', 'attach-Other', 'Female', 'AgeLess16', 'Age17-30', 'Age31-45', 'BodyFat', 'BodyNormal', 'BodyThin', 'Customer', 'Clerk', 'action-Calling', 'action-Talking', 'action-Gathering', 'action-Holding', 'action-Pusing', 'action-Pulling', 'action-CarrybyArm', 'action-CarrybyHand']


In [5]:
filename = r"../results/RAP_labels_pd.csv"
data = np.array(pd.read_csv(filename))[:, 1:]
length = len(data)
data_y = np.zeros((length, 51))
for i in range(length):
    data_y[i] = np.array(data[i, 1:52], dtype="float32")
data_y_hiar = data_y[:, list(np.hstack((low_level, mid_level, high_level)))]
split = np.load('../results/RAP_partion.npy').item()
y_test = data_y[list(split['test'][0])]
y_test_hiar = data_y_hiar[list(split['test'][0])]
print("The shape of the y_test is: ", y_test.shape)
print("The shape of the y_test is: ", y_test_hiar.shape)

The shape of the y_test is:  (8317, 51)
The shape of the y_test is:  (8317, 51)


# Positive Samples

In [6]:
for i in range(data_y_hiar.shape[1]):
    print(sum(data_y_hiar[:, i]))

39288.0
158.0
7873.0
695.0
3034.0
435.0
9198.0
5133.0
2403.0
9517.0
7185.0
13107.0
921.0
1260.0
742.0
23302.0
2708.0
1164.0
1523.0
11159.0
5544.0
12578.0
11265.0
5159.0
517.0
6543.0
754.0
2669.0
1169.0
1753.0
1146.0
526.0
1019.0
12251.0
12951.0
415.0
17882.0
23288.0
6027.0
30876.0
4064.0
39352.0
2008.0
1418.0
1364.0
3818.0
1021.0
432.0
732.0
975.0
5361.0


In [7]:
print(attributes_list[11])
print(attributes_list[24])
print(attributes_list[2])

hs-BlackHair
lb-LongTrousers
Age17-30


# Defining the function

In [15]:
def mA(y_pred, y_true):
    M = len(y_pred)
    L = len(y_pred[0])
    res = 0
    for i in range(L):
        P = sum(y_true[:, i])
        N = M - P
        TP = sum(y_pred[:, i]*y_true[:, i])
        TN = list(y_pred[:, i]+y_true[:, i] == 0).count(True)
        #print(P,',', N,',', TP,',', TN)
        if P != 0:
            res += TP/P + TN/N
        else:
            res += TN/N
    return res / (2*L)

def acc(y_pred, y_true):
    M = len(y_pred)
    M_ = 0
    res = 0
    for i in range(M):
        #print(np.shape(y_pred[i]*y_true[i]))
        if sum(y_pred[i])+sum(y_true[i])-sum(y_pred[i]*y_true[i]) != 0:
            res += sum(y_pred[i]*y_true[i]) / (sum(y_pred[i])+sum(y_true[i])-sum(y_pred[i]*y_true[i]))
            M_ += 1
    return res / M_

def prec(y_pred, y_true):
    M = len(y_pred)
    M_ = 0
    res = 0
    for i in range(M):
        if sum(y_pred[i]) != 0:
            res += sum(y_pred[i]*y_true[i]) / sum(y_pred[i])
            M_ += 1
    if M_ == 0:
        return 0
    return res / M_

def rec(y_pred, y_true):
    M = len(y_pred)
    M_ = 0
    res = 0
    for i in range(M):
        if sum(y_true[i]) != 0:
            res += sum(y_pred[i]*y_true[i]) / sum(y_true[i])
            M_ += 1
    if M_ == 0:
        return 0
    return res / M_

# Ratio>0.01

In [16]:
selected_ = []
selected = []

total = len(data_y)
print(total)
for i in range(len(attributes_list_hiar)):
    if i!=1:
        selected_.append(i)
    else:
        print(attributes_list[i])
for i in range(len(attributes_list_hiar)):
    if i!=len(low_level)+len(mid_level)+1:
        selected.append(i)
    else:
        print(attributes_list_hiar[i])
print(len(selected_))
print(len(selected))

41585
AgeLess16
AgeLess16
50
50


# Metrics

In [20]:
###hiarBayesGoogLeNet
predictions_prob = np.load("../results/predictions/hiarBayesGoogLeNet_binary51_trainable_binary51_final50iter_model_predictions_imagenet_test_RAP.npy")
predictions = np.array(predictions_prob >= 0.5, dtype="float64")#[:, selected]
label = y_test_hiar#[:, selected]
print("hiarBayesGoogLeNet")
print(mA(predictions, label))
print(acc(predictions, label))
prec_value = prec(predictions, label)
print(prec_value)
rec_value = rec(predictions, label)
print(rec_value)
print(2*prec_value*rec_value/(prec_value+rec_value))

"""
###hiarGoogLeNet
predictions_prob = np.load("../results/predictions/hiarGoogLeNet_binary51_binary51_final_model_predictions_imagenet_test_RAP.npy")
predictions = np.array(predictions_prob >= 0.5, dtype="float64")[:, selected]
label = y_test_hiar[:, selected]
print("hiarGoogLeNet")
print(mA(predictions, label))
print(acc(predictions, label))
prec_value = prec(predictions, label)
print(prec_value)
rec_value = rec(predictions, label)
print(rec_value)
print(2*prec_value*rec_value/(prec_value+rec_value))
"""

###GoogLeNet
predictions_prob = np.load("../results/predictions/GoogLeNet_binary51_binary51_b4_trainable_noweightloss__final_0partion_50iter_model_predictions_imagenet_test_RAP.npy")
predictions = np.array(predictions_prob >= 0.5, dtype="float64")#[:, selected_]
label = y_test#[:, selected_]
print("GoogLeNet")
print(mA(predictions, label))
print(acc(predictions, label))
prec_value = prec(predictions, label)
print(prec_value)
rec_value = rec(predictions, label)
print(rec_value)
print(2*prec_value*rec_value/(prec_value+rec_value))

hiarBayesGoogLeNet
0.5025885812160628
0.3685025797514278
0.7399175240614775
0.411185012868316
0.5286114605640334
GoogLeNet
0.5009783226350634
0.34691938871810485
0.529773222148778
0.48584961255996983
0.506861614231907


# Attributes

In [13]:
data_dict = {"attributes": attributes_list_hiar}

###hiarBayesGoogLeNet
predictions_prob = np.load("../results/predictions/hiarBayesGoogLeNet_binary51_binary51_final_model_predictions_imagenet_test_RAP.npy")
predictions = np.array(predictions_prob >= 0.5, dtype="float64")#[:, selected]
label = y_test_hiar#[:, selected]

print(predictions)
print(label)

metric = []
for i in range(label.shape[1]):
    #print(accuracy_score(predictions[:, i], label[:, i]))
    metric.append(accuracy_score(predictions[:, i], label[:, i]))
metric = np.asarray(metric)
data_dict.update({"hiarBayesGoogLeNet": metric})

###hiarGoogLeNet
predictions_prob = np.load("../results/predictions/hiarGoogLeNet_binary51_binary51_final_model_predictions_imagenet_test_RAP.npy")
predictions = np.array(predictions_prob >= 0.5, dtype="float64")#[:, selected]
label = y_test_hiar#[:, selected]

print(predictions)
print(label)

metric = []
for i in range(label.shape[1]):
    #print(accuracy_score(predictions[:, i], label[:, i]))
    metric.append(accuracy_score(predictions[:, i], label[:, i]))
metric = np.asarray(metric)
data_dict.update({"hiarGoogLeNet": metric})

###GoogLeNet
predictions_prob = np.load("../results/predictions/GoogLeNet_binary51_binary51_final_model_predictions_imagenet_test_RAP.npy")
predictions = np.array(predictions_prob >= 0.5, dtype="float64")#[:, selected_]
label = y_test#[:, selected_]

print(predictions)
print(label)

metric = []
for i in range(label.shape[1]):
    #print(accuracy_score(predictions[:, i], label[:, i]))
    metric.append(accuracy_score(predictions[:, i], label[:, i]))
metric = np.asarray(metric)[list(np.hstack((low_level, mid_level, high_level)))]
data_dict.update({"GoogLeNet": metric})

df = pd.DataFrame(data_dict)
df.set_index('attributes', inplace=True)
df

[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]
[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]]
[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]]
[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]]
[[0. 0. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]]


,hiarBayesGoogLeNet,hiarGoogLeNet,GoogLeNet
attributes,,,
hs-BlackHair,0.955272,0.950703,0.943249
hs-BaldHead,0.996393,0.996032,0.995672
hs-LongHair,0.915835,0.917999,0.807743
hs-Hat,0.980281,0.981844,0.980041
hs-Glasses,0.920885,0.919081,0.930504
hs-Muffler,0.990742,0.989299,0.991584
ub-Shirt,0.875195,0.891427,0.220753
ub-Sweater,0.853793,0.860286,0.878802
ub-Vest,0.974149,0.974029,0.937718


In [14]:
df.to_csv("../results/metrics/binary51_imagenet_acc7600_RAP.csv")

In [15]:
def mA_attribute(y_pred, y_true):
    M = len(y_pred)
    res = 0
    P = sum(y_true)
    N = M - P
    TP = sum(y_pred*y_true)
    TN = list(y_pred+y_true == 0).count(True)
    if P != 0:
        res += TP/P + TN/N
    else:
        res += TN/N
    return res / 2
data_dict = {"attributes": attributes_list_hiar}

###hiarBayesGoogLeNet
predictions_prob = np.load("../results/predictions/hiarBayesGoogLeNet_binary51_binary51_final_model_predictions_imagenet_test_RAP.npy")
predictions = np.array(predictions_prob >= 0.5, dtype="float64")#[:, selected]
label = y_test_hiar#[:, selected]

print(predictions)
print(label)

metric = []
for i in range(label.shape[1]):
    #print(accuracy_score(predictions[:, i], label[:, i]))
    metric.append(mA_attribute(predictions[:, i], label[:, i]))
metric = np.asarray(metric)
data_dict.update({"hiarBayesGoogLeNet": metric})

###hiarGoogLeNet
predictions_prob = np.load("../results/predictions/hiarGoogLeNet_binary51_binary51_final_model_predictions_imagenet_test_RAP.npy")
predictions = np.array(predictions_prob >= 0.5, dtype="float64")#[:, selected]
label = y_test_hiar#[:, selected]

print(predictions)
print(label)

metric = []
for i in range(label.shape[1]):
    #print(accuracy_score(predictions[:, i], label[:, i]))
    metric.append(mA_attribute(predictions[:, i], label[:, i]))
metric = np.asarray(metric)
data_dict.update({"hiarGoogLeNet": metric})

###GoogLeNet
predictions_prob = np.load("../results/predictions/GoogLeNet_binary51_binary51_final_model_predictions_imagenet_test_RAP.npy")
predictions = np.array(predictions_prob >= 0.5, dtype="float64")#[:, selected_]
label = y_test#[:, selected_]

print(predictions)
print(label)

metric = []
for i in range(label.shape[1]):
    #print(accuracy_score(predictions[:, i], label[:, i]))
    metric.append(mA_attribute(predictions[:, i], label[:, i]))
metric = np.asarray(metric)[list(np.hstack((low_level, mid_level, high_level)))]
data_dict.update({"GoogLeNet": metric})

df = pd.DataFrame(data_dict)
df.set_index('attributes', inplace=True)
df

[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]
[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]]
[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]]
[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]]
[[0. 0. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]]


,hiarBayesGoogLeNet,hiarGoogLeNet,GoogLeNet
attributes,,,
hs-BlackHair,0.639782,0.624418,0.500000
hs-BaldHead,0.652476,0.638466,0.500000
hs-LongHair,0.845684,0.858699,0.500000
hs-Hat,0.585693,0.601244,0.500000
hs-Glasses,0.612497,0.606725,0.500000
hs-Muffler,0.542069,0.562588,0.500000
ub-Shirt,0.797140,0.814505,0.500428
ub-Sweater,0.643136,0.631008,0.500000
ub-Vest,0.833022,0.824848,0.500000


In [16]:
df.to_csv("../results/metrics/binary51_imagenet_ma7600_RAP.csv")

# In HiarBayesGoogLeNet, Select one each level

In [21]:
###hiarBayesGoogLeNet
predictions_prob = np.load("../results/predictions/hiarBayesGoogLeNet_binary3_binary3_epoch50_valloss0.40.h_predictions_imagenet_test_RAP.npy")
predictions = np.array(predictions_prob >= 0.5, dtype="float64")#[:, selected]
label = y_test[:, [11,24,3]]

print(predictions)
print(label)

for i in range(label.shape[1]):
    print(accuracy_score(predictions[:, i], label[:, i]))
    
for i in range(label.shape[1]):
    print(mA_attribute(predictions[:, i], label[:, i]))

[[1. 1. 0.]
 [1. 0. 1.]
 [1. 1. 0.]
 ...
 [1. 1. 0.]
 [1. 1. 0.]
 [1. 0. 1.]]
[[1. 1. 1.]
 [1. 0. 0.]
 [1. 1. 1.]
 ...
 [0. 1. 1.]
 [1. 0. 0.]
 [1. 0. 0.]]
0.9478177227365636
0.8421305759288205
0.3824696404953709
0.6308526698426072
0.8337924336926777
0.39862568115648445


In [22]:
print(mA(predictions, label))
print(acc(predictions, label))
prec_value = prec(predictions, label)
print(prec_value)
rec_value = rec(predictions, label)
print(rec_value)
print(2*prec_value*rec_value/(prec_value+rec_value))

0.6210902615639231
0.6732812186811066
0.8233486017357763
0.8115317720341617
0.817397481171928


# In GoogLeNet, Select one each level

In [29]:
###hiarBayesGoogLeNet
predictions_prob = np.load("../results/predictions/GoogLeNet_binary51_binary51_b4_mysplit_final_model_predictions_imagenet_test_RAP.npy")
predictions = np.array(predictions_prob >= 0.5, dtype="float64")[:, selected_]
label = y_test[:, selected_]

print(predictions)
print(label)

for i in range(label.shape[1]):
    print(accuracy_score(predictions[:, i], label[:, i]))
    
for i in range(label.shape[1]):
    print(mA_attribute(predictions[:, i], label[:, i]))

[[0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 1. ... 0. 0. 0.]
 [1. 1. 0. ... 0. 0. 0.]
 [1. 1. 0. ... 0. 0. 0.]]
0.6878682217145605
0.5467115546471083
0.4471564265961284
0.856679091018396
0.5901166285920404
0.8986413370205603
0.945653480822412
0.9503426716364074
0.9956715161716966
0.8077431766261873
0.9183599855717206
0.9800408801250451
0.9305037874233497
0.9915835036671877
0.7827341589515451
0.8788024528075027
0.9377179271371889
0.7699891787904293
0.8291451244439101
0.35241072502104126
0.9758326319586389
0.9693399062161837
0.9822051220391969
0.5016231814356138
0.9333894433088854
0.9734279187206926
0.9604424672357821
0.7321149452927739
0.8660574726463869
0.7010941445232656
0.7271852831549838
0.8743537333173019
0.9877359624864734
0.8470602380666106
0.9823253577010942
0.939280990741854
0.9722

In [30]:
print(mA(predictions, label))
print(acc(predictions, label))
prec_value = prec(predictions, label)
print(prec_value)
rec_value = rec(predictions, label)
print(rec_value)
print(2*prec_value*rec_value/(prec_value+rec_value))

0.49995229334997476
0.32598183494673033
0.7176405755280579
0.36504557924541114
0.48392882541009785
